In [5]:
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import cv2
import os
import pandas as pd

#### Load Data, Label for prediction (currently use one dataset (Brain MRI Data2))

In [82]:
labels = ["glioma_tumor", "meningioma_tumor", "no_tumor", "pituitary_tumor"]
sets = ["Training", "Testing"]
image_set = list()
label_set = list()

for s in sets:
    for label in tqdm(labels):
        path = os.path.join("Brain_MRI_data2", s, label)
        images = os.listdir(path)
        for image in images:
            current_image = cv2.imread(os.path.join(path, image))
            current_image = cv2.resize(current_image, (224, 224))
            image_set.append(current_image)
            label_set.append(label)

features = np.array(image_set)
labels = pd.get_dummies(np.array(label_set)).values.argmax(1)

100%|██████████| 4/4 [00:00<00:00, 11.25it/s]


In [83]:
features.shape

(3264, 224, 224, 3)

#### split dataset into train and test

In [84]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.25, random_state=42)

#### Use a simple CNN for training

In [91]:
import tensorflow as tf
from keras import layers, models
model = models.Sequential()
model.add(layers.Conv2D(16, 3, activation='relu', input_shape=(224, 224, 3)))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(32, 3, activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(64, 3, activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(4))

In [92]:
model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_42 (Conv2D)          (None, 222, 222, 16)      448       
                                                                 
 max_pooling2d_40 (MaxPoolin  (None, 111, 111, 16)     0         
 g2D)                                                            
                                                                 
 conv2d_43 (Conv2D)          (None, 109, 109, 32)      4640      
                                                                 
 max_pooling2d_41 (MaxPoolin  (None, 54, 54, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_44 (Conv2D)          (None, 52, 52, 64)        18496     
                                                                 
 max_pooling2d_42 (MaxPoolin  (None, 26, 26, 64)     

In [93]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [94]:
epochs=20
history = model.fit(x=X_train,
                    y=y_train,
                    batch_size=8,
                    epochs=epochs,
                    validation_data=(X_test, y_test))

Epoch 1/20
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Inconsistent ASTs detected. This is a bug. Cause: 
inconsistent values for field args: [<gast.gast.Name object at 0x2cfc595b0>] and []Diff:
*** Original nodes

--- Reparsed nodes

***************

*** 30,68 ****

  | | | ]
  | | | value=Constant:
  | | | | value=None
  | | | | kind=None
  | | FunctionDef:
  | | | name=u"inner_factory"
  | | | args=arguments:
! | | | | args=[
  | | | | | Name:
  | | | | | | id=u"ag__"
  | | | | | | ctx=Param()
  | | | | | | annotation=None
  | | | | | | type_comment=None
  | | | | ]
- | | | | posonlyargs=[]
  | | | | vararg=None
  | | | | kwonlyargs=[]
  | | | | kw_defaults=[]
  | | | | kwarg=None
  | | | | defaults=[]
  | | | body=[
  | | | | FunctionDef:
  | | | | | name=u"tf__train_function"
  | | | | | args=arguments:
! | | | | | | args=[
  | | | | | | | Name:
  | | | | | | 

#### Using Resnet50 for training

In [88]:
model_restnet = tf.keras.applications.resnet.ResNet50()

In [89]:
model_restnet.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [90]:
model_restnet.fit(x=X_train,
                    y=y_train,
                    epochs=epochs,
                    validation_data=(X_test, y_test))

Epoch 1/20
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Inconsistent ASTs detected. This is a bug. Cause: 
inconsistent values for field args: [<gast.gast.Name object at 0x2bef280a0>] and []Diff:
*** Original nodes

--- Reparsed nodes

***************

*** 30,68 ****

  | | | ]
  | | | value=Constant:
  | | | | value=None
  | | | | kind=None
  | | FunctionDef:
  | | | name=u"inner_factory"
  | | | args=arguments:
! | | | | args=[
  | | | | | Name:
  | | | | | | id=u"ag__"
  | | | | | | ctx=Param()
  | | | | | | annotation=None
  | | | | | | type_comment=None
  | | | | ]
- | | | | posonlyargs=[]
  | | | | vararg=None
  | | | | kwonlyargs=[]
  | | | | kw_defaults=[]
  | | | | kwarg=None
  | | | | defaults=[]
  | | | body=[
  | | | | FunctionDef:
  | | | | | name=u"tf__train_function"
  | | | | | args=arguments:
! | | | | | | args=[
  | | | | | | | Name:
  | | | | | | 

/opt/anaconda3/envs/CS_HW/lib/python3.8/site-packages/keras/backend.py:5582: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


77/77 [==============================] - ETA: 0s - loss: 0.8788 - accuracy: 0.7525WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x2ebd9f4c0> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Inconsistent ASTs detected. This is a bug. Cause: 
inconsistent values for field args: [<gast.gast.Name object at 0x2ff4e1220>] and []Diff:
*** Original nodes

--- Reparsed nodes

***************

*** 30,68 ****

  | | | ]
  | | | value=Constant:
  | | | | value=None
  | | | | kind=None
  | | FunctionDef:
  | | | name=u"inner_factory"
  | | | args=arguments:
! | | | | args=[
  | | | | | Name:
  | | | | | | id=u"ag__"
  | | | | | | ctx=Param()
  | | | | | | annotation=None
  | | | | | | type_comment=None
  | | | | ]
- | | | | posonlyargs=[]
  | | | | vararg=None
  | | | | kwonlyargs=[]
  | | | | kw_